In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
import warnings 
warnings.filterwarnings('ignore')

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv0 = nn.Conv2d(1, 8, 3, padding=1) #input -? OUtput? RF
        self.bn0 = nn.BatchNorm2d(8)
        self.conv01 = nn.Conv2d(8, 8, 3, padding=1)
        self.bn01 = nn.BatchNorm2d(8)
        self.pool0 = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout(0.1)
        
        self.conv1 = nn.Conv2d(8, 16, 3, padding=1) #input -? OUtput? RF
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.drop2 = nn.Dropout(0.1)

        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(32)
        self.avgpool = nn.AvgPool2d(7)
        self.drop3 = nn.Dropout(0.1)        

        self.fc1 = nn.Linear(32,16)
        self.fc2 = nn.Linear(16,10)


    def forward(self, x):
        x = self.pool0(self.bn01(F.relu(self.conv01(self.bn0(F.relu(self.conv0(x)))))))
        x=self.drop1(x)
        x = self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x)))))))
        x=self.drop2(x)
        x = self.avgpool(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x)))))))
        x=self.drop3(x)
        x = x.view(-1, 32)
        x = self.fc2(self.fc1(x))
        return F.log_softmax(x)

In [13]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3            [-1, 8, 28, 28]             584
       BatchNorm2d-4            [-1, 8, 28, 28]              16
         MaxPool2d-5            [-1, 8, 14, 14]               0
           Dropout-6            [-1, 8, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]           1,168
       BatchNorm2d-8           [-1, 16, 14, 14]              32
            Conv2d-9           [-1, 16, 14, 14]           2,320
      BatchNorm2d-10           [-1, 16, 14, 14]              32
        MaxPool2d-11             [-1, 16, 7, 7]               0
          Dropout-12             [-1, 16, 7, 7]               0
           Conv2d-13             [-1, 32, 7, 7]           4,640
      BatchNorm2d-14             [-1, 3

In [14]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [15]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    correct = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        train_pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += train_pred.eq(target.view_as(train_pred)).sum().item()

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nEpoch: {}, Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        epoch, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [16]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Epoch: 1, Test set: Average loss: 0.1101, Accuracy: 9665/10000 (96.65%)



  0%|          | 0/469 [00:00<?, ?it/s]


Epoch: 2, Test set: Average loss: 0.0410, Accuracy: 9866/10000 (98.66%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Epoch: 18, Test set: Average loss: 0.0178, Accuracy: 9942/10000 (99.42%)



  0%|          | 0/469 [00:00<?, ?it/s]


Epoch: 19, Test set: Average loss: 0.0188, Accuracy: 9940/10000 (99.40%)



loss=0.0025993806775659323 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.49it/s]



Epoch: 20, Test set: Average loss: 0.0178, Accuracy: 9944/10000 (99.44%)

